In [1]:
#Determine what platform and the path to data
import platform
import numpy as np
import pandas as pd

#Predetermine the column types, saves on any transformtions later on. 
#Currently pd.Categorical is not a valid dtype so they come in as objects. Ned to convert
colTypes = {'Elevation':np.float64,'Aspect':np.float64,'Slope':np.float64,'HD.Hydro':np.float64,'VD.Hyrdro': np.float64,
            'HD.Road': np.float64, 'HS.9am':np.float64,'HS.noon':np.float64,'HS.3pm':np.float64,
            'wildernessArea':pd.Categorical,'soilType':pd.Categorical,'climaticZone':pd.Categorical,
            'geologicZone':pd.Categorical,'coverType':pd.Categorical} 




if platform.system()  == "Windows":
     pathToReadData =  "C:\\Users\\IBM_ADMIN\\Desktop\\OneDrive\\PublicData\\AM\\v2\\"
     pathToWriteData = "C:\\Users\\IBM_ADMIN\\Desktop\\OneDrive\\PublicData\\AM\\v3\\"
elif platform.system() == "Darwin":
     pathToReadData = "/Users/briancarter1/Desktop/OneDrive/PublicData/AM/v2/"
     pathToWriteData = "/Users/briancarter1/Desktop/OneDrive/PublicData/AM/v3/"

In [2]:
#Read in the data without binary encoding, 15 original columns 
import pandas as pd
coverData = pd.read_csv(pathToReadData+"covtype15.csv",dtype=colTypes )
coverData.head()


,Elevation,Aspect,Slope,HD.Hydro,VD.Hydro,HD.Road,HD.Fire,HS.9am,HS.noon,HS.3pm,wildernessArea,soilType,climaticZone,geologicZone,coverType
0,3391,209,12,558,174,3306,211,251,173,552,ComanchePeak,Bross,alpine,igneous and metamorphic,1.SpruceFir
1,3374,199,17,499,168,3276,212,252,170,576,ComanchePeak,Bross,alpine,igneous and metamorphic,1.SpruceFir
2,3366,197,17,484,160,3252,213,252,168,600,ComanchePeak,Bross,alpine,igneous and metamorphic,1.SpruceFir
3,3357,196,16,469,151,3228,214,252,167,624,ComanchePeak,Bross,alpine,igneous and metamorphic,1.SpruceFir
4,3346,206,17,426,140,3223,207,253,177,633,ComanchePeak,Bross,alpine,igneous and metamorphic,1.SpruceFir


In [3]:
cat_columns = ['wildernessArea','soilType','climaticZone','geologicZone']
wilderness = list(coverData['wildernessArea'].unique())
soil = list(coverData['soilType'].unique())
climaticZone = list(coverData['climaticZone'].unique())
geo = list(coverData['geologicZone'].unique())
cover = list(coverData['coverType'].unique())

coverData['wildernessArea'] = pd.Categorical(coverData['wildernessArea'], categories=wilderness)
coverData['soilType'] = pd.Categorical(coverData['soilType'], categories=soil)
coverData['climaticZone'] = pd.Categorical(coverData['climaticZone'], categories=climaticZone)
coverData['geologicZone'] = pd.Categorical(coverData['geologicZone'], categories=geo)
coverData['coverType'] = pd.Categorical(coverData['coverType'], categories=cover)

In [4]:

import sklearn.cross_validation as cv

#X = coverData.ix[:, coverData.columns != 'coverType']
# .ix makes no data.frame categorical variables are reset to objects as not valid dtype


#Used later
targetLabels=['1.Spruce','2.Lodge','3.Ponder','4.Cotton','5.Aspen','6.Douglas','7.Krymmholz']
targetWeights={0 : 0.365, 1 : 0.498, 2 :0.062, 3 : 0.005, 4 : 0.016, 5 : 0.03, 6: 0.035}

X = coverData.drop('coverType', axis=1)


from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(coverData['coverType'])
y=le.transform(coverData['coverType'])

In [5]:
import sklearn.cross_validation as cv
X_train, X_test, y_train, y_test = cv.train_test_split(X,y,test_size=0.9,random_state=79)

In [6]:
from sklearn import tree

In [9]:
X_train_dummies = pd.get_dummies(X_train)

print (X_train.shape)
print (X_train_dummies.shape)

clf = tree.DecisionTreeClassifier(criterion='gini',max_depth=5,min_samples_split=20).fit(X_train_dummies,y_train)

(58101, 14)
(58101, 65)


In [12]:
from sklearn.externals.six import StringIO  
import pydot 
dot_data = StringIO() 
tree.export_graphviz(clf, out_file=dot_data,feature_names=X_train_dummies.columns) 
graph = pydot.graph_from_dot_data(dot_data.getvalue()) 
graph.write_pdf("hope.pdf") 

True